In [38]:
import pandas as pd
import talib
from enum import Enum
import QuantLib as ql
from market import MarketData
from datetime import date 

class OrderType(Enum):
    BUY = "buy"
    SELL = "sell"

class PositionType(Enum):
    LONG = 1
    SHORT = -1

class OptionType(Enum):
    CALL = "Call"
    PUT = "Put"

class AssetType(Enum):
    EQUITY = 'equity'
    BOND = "bond"
    OPTION = "option"

# CORE

In [2]:
class Indicator:
    def __init__(self):
        self.methods = {}

    def register(self, name=None):
        """
        A decorator to register an indicator method with a custom or derived name.
        """
        def decorator(func):
            indicator_name = name or func.__name__
            self.methods[indicator_name] = func
            return func
        return decorator

    def compute(self, name, *args, **kwargs):
        """
        Compute the registered indicator.
        """
        if name not in self.methods:
            raise ValueError(f"Indicator '{name}' is not registered.")
        return self.methods[name](*args, **kwargs)


class Backtester:
    def __init__(self, history: pd.DataFrame, starting_balance: float,):
        self.starting_balance = starting_balance
        self.position_open = False
        self.equity_history = []
        self.signals = []
        self.indicators = Indicator()  # Indicator object for registration
        self._add_data(history)
        self.indicator_values = pd.DataFrame(index=history.index)

    def _add_data(self, history):
        self._history = history

    def backtest(self):
        """ chiama in automatico on_data a ogni iterazione e aggiorna history con i valori noti fino alla data"""
        for i in range(0, len(self._history)):
            self.history = self._history.iloc[:i + 1]
            self.on_data()

    def add_indicator(self, name, value):
        """
        Automatically updates the indicator DataFrame.
        """
        self.indicator_values.loc[self.history.index[-1], name] = value

    def get_data_at_index(self, idx):
        """
        Returns the historical data and indicators up to the specified index.
        :param idx: The index yyyy-mm-dd up to which to return data.
        :return: A combined DataFrame with historical data and indicators.
        """
        if idx not in self._history.index:
            raise ValueError(f"Index {idx} is not in the historical data.")
        
        # Slice the historical data and indicators up to the specified index
        data = self._history.loc[:idx]
        indicators = self.indicator_values.loc[:idx]
        
        # Combine historical data and indicators into one DataFrame
        return pd.concat([data, indicators], axis=1)
    
    def execute_trade(self, trade_type, quantity, price):
        """Esegue operazioni di acquisto o vendita."""
        cost = quantity * price

        if trade_type == OrderType.BUY:
            if self.starting_balance >= cost:
                self.starting_balance -= cost
                self.position_open = True
                self.signals.append({"type": "BUY", "price": price})
            else:
                print("Fondi insufficienti per comprare.")
        elif trade_type == OrderType.SELL:
            if self.position_open:
                self.starting_balance += cost
                self.position_open = False
                self.signals.append({"type": "SELL", "price": price})
            else:
                print("Nessuna posizione aperta da vendere.")

        # Aggiorna l'equity history
        self.equity_history.append(self.starting_balance)

    def on_data(self):
        """
        Implement trading logic.
        """
        pass


class BacktesterConcrete(Backtester):
    def __init__(self, history, starting_balance, ema_window=20):
        super().__init__(history, starting_balance)
        self.ema_window = ema_window

        # Register EMA indicator
        @self.indicators.register(name="ema")
        def ema(series, window):
            return talib.EMA(series, timeperiod=window)

    def on_data(self):
        """Calcola l'EMA e genera segnali di trading."""
        latest_price = self.history["price"].iloc[-1]

        # Calcola EMA
        ema_series = self.indicators.compute("ema", self.history["price"], self.ema_window)
        latest_ema = ema_series.iloc[-1] if not ema_series.isna().iloc[-1] else None

        # Salva EMA nel DataFrame indicator_values
        self.add_indicator("ema", latest_ema)

        # Genera segnali di trading
        if latest_ema is not None:
            if latest_price > latest_ema and not self.position_open:
                self.execute_trade(OrderType.BUY, quantity=1, price=latest_price)
            elif latest_price < latest_ema and self.position_open:
                self.execute_trade(OrderType.SELL, quantity=1, price=latest_price)


# Position and Portfolio

In [220]:
from abc import ABC, abstractmethod
from datetime import date 

class Position(ABC):
    def __init__(self, symbol: str, asset_type: AssetType, quantity: float, market_data: dict, position_type: PositionType):
        self.symbol = symbol
        self.asset_type = asset_type  # "bond", "equity", "option"
        self.quantity = quantity
        self.trade_date = market_data['ref_date']
        self.entry_price = self.calculate_value(market_data) 
        self.position_type = position_type

    @abstractmethod
    def calculate_value(self, market_data: dict) -> float:
        """Metodo astratto per calcolare il valore della posizione."""
        pass

    def calculate_pnl(self, market_data: dict) -> float:
        """Metodo astratto per calcolare il valore della posizione."""
        return (self.calculate_value(market_data) - self.entry_price) * self.position_type.value * self.quantity 


class EquityPosition(Position):
    def __init__(self, symbol: str, quantity: float, market_data: dict, position_type: PositionType):
        super().__init__(symbol, AssetType.EQUITY, quantity, market_data, position_type)

    def calculate_value(self, market_data: dict) -> float:
        """Usa il prezzo corrente dell'azione per calcolare il valore della posizione."""
        return market_data[self.asset_type.value][self.symbol]["price"]


class OptionPosition(Position):
    def __init__(self, symbol: str, quantity: float, market_data: dict, strike_price: float, expiry_date: date, option_type: OptionType, position_type: PositionType,
                 day_count = ql.Actual365Fixed(), calendar = ql.TARGET()):
        self.strike_price = strike_price
        self.expiry_date = expiry_date
        if option_type == OptionType.CALL:
            self.option_type = ql.Option.Call
        else:
            self.option_type = ql.Option.Put
        self.day_count = day_count
        self.calendar = calendar
        self.option = self._build_option()
        super().__init__(symbol, AssetType.OPTION, quantity, market_data, position_type)

    def calculate_value(self, market_data: dict):
        valuation_date = ql.Date.from_date(market_data['ref_date'])
        ql.Settings.instance().evaluationDate = valuation_date
        
        maturity_date = ql.Date.from_date(self.expiry_date)
        sigma = market_data['volatility'][self.symbol].blackVol(maturity_date, self.strike_price)

        spot_handle = ql.QuoteHandle(
        ql.SimpleQuote(market_data['equity'][self.symbol]['price'])
        )
        flat_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(valuation_date, market_data['rate']['riskfree'], self.day_count)
        )
        dividend_yield = ql.YieldTermStructureHandle(
        ql.FlatForward(valuation_date, market_data['equity'][self.symbol]['div_yield'], self.day_count)
        )
        flat_vol_ts = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(valuation_date, self.calendar, sigma, self.day_count)
        )
        bsm_process = ql.BlackScholesMertonProcess(spot_handle,
                                                dividend_yield,
                                                flat_ts,
                                                flat_vol_ts)

        self.option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
        return self.option.NPV()
    
    def _build_option(self):
        maturity_date = ql.Date.from_date(self.expiry_date)
        payoff = ql.PlainVanillaPayoff(self.option_type, self.strike_price)
        exercise = ql.EuropeanExercise(maturity_date)
        return ql.VanillaOption(payoff, exercise)


    

class Portfolio:
    def __init__(self, initial_balance: float):
        self.cash_balance = initial_balance  # Saldo disponibile
        self.positions = []  # Lista di posizioni aperte

    def add_position(self, position):
        """Aggiunge una posizione al portafoglio."""
        cost = position.entry_price * position.quantity
        
        if self.cash_balance >= cost:
            self.positions.append(position)
            self.cash_balance -= cost  # Aggiorna il saldo
        else:
            print("Fondi insufficienti per aprire la posizione.")

    def close_position(self, position, market_data: dict):
        """Chiude una posizione esistente e aggiorna il saldo."""
        if position in self.positions:
            value = position.calculate_value(market_data)
            self.cash_balance += value  # Aggiorna il saldo con il valore di vendita
            self.positions.remove(position)
        else:
            print("Posizione non trovata nel portafoglio.")

    def total_value(self, market_data: dict):
        """
        Calcola il valore totale del portafoglio, considerando le posizioni aperte.
        :param market_data: dizionario con i dati di mercato {"equity": {...}, "rates": {...}, "volatilities": {...}}
        :return: valore totale del portafoglio
        """
        total = self.cash_balance  # Include il saldo disponibile
        for position in self.positions:
            total += position.calculate_value(market_data)
        return total

    def summary(self):
        """Mostra un riepilogo del portafoglio."""
        print(f"Saldo disponibile: {self.cash_balance:.2f}")
        print("Posizioni aperte:")
        for position in self.positions:
            print(f"- {position.asset_type} | Quantità: {position.quantity} | Prezzo entrata: {position.entry_price}")



# debug singoli

## pricing

In [122]:
symbol = 'ctp1'
start_date = date(2022, 10, 14)
end_date = date(2023, 10, 12)
market = MarketData(start_date, end_date)

market.build(symbol)

equity

In [216]:
entry_mkt = market.market_data[date(2022, 10, 14)]
exit_mkt = market.market_data[date(2022, 11, 14)]

equity_position = EquityPosition(symbol, quantity=10, market_data=entry_mkt, position_type=PositionType.LONG)
print("entry ref: ", entry_mkt['equity'][symbol]['price'])

entry ref:  1.7124


In [217]:
print("exit ref: ", exit_mkt['equity'][symbol]['price'])
print("PNL :", equity_position.calculate_pnl(exit_mkt))


exit ref:  2.1865
PNL : 4.741000000000001


option

In [221]:
quantity= 1
market_data= entry_mkt
strike_price= 1.7
expiry_date= date(2025, 1, 20)
option_type= OptionType.CALL
position_type= PositionType.LONG

In [222]:
opt = OptionPosition(symbol,
               10,
               entry_mkt,
               strike_price,
               expiry_date,
               option_type,
               position_type)

In [223]:
opt.calculate_pnl(exit_mkt)

3.880492462894305

In [ ]:
opt.calculate_value(exit_mkt)

In [227]:
opt.calculate_value(entry_mkt)

0.15060245190513466

In [228]:
opt.calculate_value(exit_mkt)

0.5386516981945652

# Backtest

In [26]:
# params 
df_prices = pd.read_csv('data/prices/isp_prices.csv')
params = {
    "starting_balance": 10000,
    "ema_window": 20
}

In [22]:
backtester  = BacktesterConcrete(df_prices, **params)
backtester.backtest()